In [1]:
%matplotlib inline

# モデルの保存と読み込み

このユニットでは、モデル予測の保存、読み込み、実行によってモデルの状態を持続させる方法を見ていきます。

In [2]:
import torch
import torch.onnx as onnx
import torchvision.models as models

## モデルウェイトの保存と読み込み

PyTorchのモデルは、学習したパラメータを内部の状態辞書（`state_dict`と呼びます）に保存します。これらは `torch.save` メソッドで永続化することができます。

In [3]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'data/model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/shogo/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 63.2MB/s]


モデルのウェイトをロードするには、まず同じモデルのインスタンスを作成し、`load_state_dict()`メソッドを使ってパラメータをロードする必要があります。

In [4]:
model = models.vgg16()  # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('data/model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

> **Note:** 推論を行う前に、必ず`model.eval()`メソッドを呼び出し、ドロップアウトとバッチ正規化のレイヤーを評価モードに設定してください。これを行わないと、推論結果に一貫性がなくなります。

## シェイプ付きモデルの保存と読み込み

モデルの重みを読み込む際には、まずモデルクラスをインスタンス化する必要がありました。モデルクラスはネットワークの構造を定義しているからです。このクラスの構造をモデルと一緒に保存したい場合は、`model`（`model.state_dict()`ではなく）を保存関数に渡します。

In [5]:
torch.save(model, 'data/vgg_model.pth')

そして、次のようにモデルを読み込みます。

In [6]:
model = torch.load('data/vgg_model.pth')

> **Note:** この方法は、モデルをシリアル化する際にPythonの[pickle](https://docs.python.org/3/library/pickle.html)モジュールを使用するため、モデルをロードする際に実際のクラス定義が利用可能であることに依存します。

## モデルのONNXへのエクスポート

PyTorchは、ONNXへのエクスポートをネイティブにサポートしています。しかし、PyTorch の実行グラフは動的であるため、エクスポート処理では実行グラフを横断して ONNX モデルを生成する必要があります。このため、適切なサイズのテスト変数をエクスポートルーチンに渡す必要があります（ここでは、適切なサイズのダミーのゼロテンソルを作成します）。

In [7]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'data/model.onnx')

ONNXモデルでは、異なるプラットフォームや異なるプログラミング言語で推論を実行するなど、様々なことが可能です。詳しくは、[ONNXチュートリアル](https://github.com/onnx/tutorials)をご覧ください。

おめでとうございます。これでPyTorchの初心者向けチュートリアルは終了です。このチュートリアルが、PyTorchで深層学習を始める際のお役に立てれば幸いです。

# 自分の知識を確認する

1. PyTorchモデル`state_dict`とは？

- モデルの内部状態を表す辞書で、現在の精度と損失の値が格納されています。
  > 不正解
- 学習に使用したデータのバージョンを格納するモデルの内部状態辞書です。
  > 不正解
- 内部レイヤーを格納するモデルの内部状態辞書です。
  > 不正解
- 学習されたパラメータを格納するモデルの内部状態辞書です。
  > 正解
